<a href="https://colab.research.google.com/github/4sz5sz6sz/AI-Text-Classifier-DACON2025/blob/main/improved_tfidf_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF-IDF + XGBoost 개선 실험

**개선 사항:**
- 하이퍼파라미터 튜닝 추가
- 교차 검증 적용
- 피처 개수 조정
- 성능 비교 분석

## 🚀 Google Colab 실행 가이드

## 파일 업로드 방법

**대용량 파일(500MB+)**: Google Drive 공개 링크 사용 (아래 코드 참고)
**소용량 파일**: 직접 업로드

**주의사항:**
- 파일 업로드는 한 번만 하면 됩니다
- 런타임이 재시작되면 다시 업로드해야 합니다
- 인코딩 오류가 발생하면 자동으로 여러 방식을 시도합니다

**실행 순서:**
1. 모든 셀을 순서대로 실행
2. 파일 업로드가 요청되면 데이터 파일들을 업로드
3. 최종 결과는 자동으로 다운로드됩니다

# Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Data Load & Split

In [ ]:
# 🔗 대용량 파일 자동 다운로드
import gdown
import os

# 파일 ID 설정 (Google Drive 공개 링크에서 추출)
TRAIN_FILE_ID = "1teA9GmYlIsutaDLWvCCsLeh7833t-TC_"
TEST_FILE_ID = "1bGC_YWtNUOroHARfmzCjrL8oPcvb7Tpw"
SAMPLE_FILE_ID = "1ebrHVj-CtM-7aEz4OqP-bCHlazle-PKM"

def download_from_drive(file_id, filename):
    if file_id and not os.path.exists(filename):
        url = f'https://drive.google.com/uc?id={file_id}'
        print(f"📥 {filename} 다운로드 중...")
        gdown.download(url, filename, quiet=False)
        print(f"✅ {filename} 다운로드 완료!")
    elif os.path.exists(filename):
        print(f"✅ {filename} 이미 존재함")

# 자동 다운로드 실행
download_from_drive(TRAIN_FILE_ID, 'train.csv')
download_from_drive(TEST_FILE_ID, 'test.csv')
download_from_drive(SAMPLE_FILE_ID, 'sample_submission.csv')

In [ ]:
import pandas as pd
import os
import gc
import psutil

# 메모리 사용량 확인 함수
def get_memory_usage():
    """현재 메모리 사용량 확인"""
    process = psutil.Process(os.getpid())
    memory_mb = process.memory_info().rss / 1024 / 1024
    return memory_mb

print(f"🔍 시작 시 메모리 사용량: {get_memory_usage():.1f}MB")

# 메모리 효율적인 CSV 읽기 함수
def read_csv_robust(file_path):
    """메모리 효율적이고 다양한 인코딩으로 CSV 파일을 읽는 함수"""
    encodings = ['utf-8', 'utf-8-sig', 'cp949', 'euc-kr']
    
    print(f"📂 {file_path} 로딩 중...")
    start_memory = get_memory_usage()
    
    for encoding in encodings:
        try:
            # 메모리 효율적 로딩 설정
            df = pd.read_csv(
                file_path, 
                encoding=encoding,
                dtype={'id': 'int32'},  # ID는 int32로 충분
                low_memory=False        # 타입 추론 개선
            )
            
            end_memory = get_memory_usage()
            memory_used = end_memory - start_memory
            
            print(f"✅ {file_path} 읽기 성공")
            print(f"   - 인코딩: {encoding}")
            print(f"   - 메모리 사용: +{memory_used:.1f}MB")
            print(f"   - 데이터 크기: {df.shape}")
            
            return df
        except Exception as e:
            continue
    
    # 모든 인코딩 실패 시 에러 무시 모드
    try:
        df = pd.read_csv(file_path, encoding='utf-8', errors='ignore', low_memory=False)
        end_memory = get_memory_usage()
        memory_used = end_memory - start_memory
        
        print(f"✅ {file_path} 읽기 성공 (에러 무시 모드)")
        print(f"   - 메모리 사용: +{memory_used:.1f}MB")
        return df
    except Exception as e:
        print(f"❌ {file_path} 읽기 실패: {e}")
        raise

# 데이터 파일 읽기
print("📁 데이터 파일 로딩 시작...")
gc.collect()  # 사전 메모리 정리

try:
    train = read_csv_robust('train.csv')
    test = read_csv_robust('test.csv')
    sample_submission = read_csv_robust('sample_submission.csv')
    
    print(f"\n📊 데이터 로딩 완료!")
    print(f"✅ Train: {train.shape}")
    print(f"✅ Test: {test.shape}")
    print(f"✅ Sample submission: {sample_submission.shape}")
    print(f"🧠 총 메모리 사용량: {get_memory_usage():.1f}MB")
    
    # 데이터 타입 최적화 (메모리 절약)
    for df_name, df in [('train', train), ('test', test)]:
        if 'id' in df.columns:
            df['id'] = df['id'].astype('int32')
        print(f"📝 {df_name} 데이터 타입 최적화 완료")
    
    gc.collect()
    print("🧹 메모리 정리 완료")
    
except Exception as e:
    print(f"❌ 데이터 로딩 중 오류: {e}")
    print("💡 해결책:")
    print("1. 파일이 올바르게 다운로드되었는지 확인")
    print("2. 런타임 재시작 후 다시 시도")
    raise

=== train.csv 읽기 ===
'utf-8-sig' 인코딩으로 시도 중...
'utf-8-sig' 인코딩에서 다른 오류: Error tokenizing data. C error: out of memory
'utf-8' 인코딩으로 시도 중...
'utf-8-sig' 인코딩에서 다른 오류: Error tokenizing data. C error: out of memory
'utf-8' 인코딩으로 시도 중...
성공! 'utf-8' 인코딩으로 파일을 읽었습니다.
Train 데이터 shape: (97172, 3)

=== test.csv 읽기 ===
'utf-8-sig' 인코딩으로 시도 중...
성공! 'utf-8-sig' 인코딩으로 파일을 읽었습니다.
Test 데이터 shape: (1962, 4)
성공! 'utf-8' 인코딩으로 파일을 읽었습니다.
Train 데이터 shape: (97172, 3)

=== test.csv 읽기 ===
'utf-8-sig' 인코딩으로 시도 중...
성공! 'utf-8-sig' 인코딩으로 파일을 읽었습니다.
Test 데이터 shape: (1962, 4)


In [ ]:
#train = pd.read_csv('./train.csv', encoding='utf-8-sig')
#test = pd.read_csv('./test.csv', encoding='utf-8-sig')

# 데이터 확인
try:
    print("=== 데이터 기본 정보 ===")
    print(f"Train shape: {train.shape}")
    print(f"Test shape: {test.shape}")
    
    # 컬럼 확인
    print(f"\nTrain 컬럼: {train.columns.tolist()}")
    print(f"Test 컬럼: {test.columns.tolist()}")
    
    # target 변수 확인
    if 'generated' in train.columns:
        print(f"\nGenerated 비율: {train['generated'].mean():.3f}")
        print(f"Generated 값 분포:")
        print(train['generated'].value_counts())
    else:
        print("\n⚠️ 'generated' 컬럼을 찾을 수 없습니다.")
        print("사용 가능한 컬럼:", train.columns.tolist())
    
    # 데이터 미리보기
    print("\n=== Train 데이터 미리보기 ===")
    print(train.head())
    
    print("\n=== Test 데이터 미리보기 ===")
    print(test.head())
    
    # 결측값 확인
    print("\n=== 결측값 확인 ===")
    print("Train 결측값:")
    print(train.isnull().sum())
    print("\nTest 결측값:")
    print(test.isnull().sum())
    
except NameError:
    print("❌ 데이터가 로드되지 않았습니다. 위의 셀을 먼저 실행해주세요.")
except Exception as e:
    print(f"❌ 데이터 확인 중 오류: {e}")

Train shape: (97172, 3)
Test shape: (1962, 4)
Generated 비율: 0.082


In [ ]:
# 데이터 전처리 및 분할
try:
    # 테스트 데이터 컬럼명 통일 (필요한 경우)
    if 'paragraph_text' in test.columns and 'full_text' not in test.columns:
        test = test.rename(columns={'paragraph_text': 'full_text'})
        print("✅ Test 데이터의 'paragraph_text' 컬럼을 'full_text'로 변경했습니다.")
    
    # 필요한 컬럼 확인
    required_train_cols = ['title', 'full_text', 'generated']
    required_test_cols = ['title', 'full_text']
    
    missing_train_cols = [col for col in required_train_cols if col not in train.columns]
    missing_test_cols = [col for col in required_test_cols if col not in test.columns]
    
    if missing_train_cols:
        print(f"❌ Train 데이터에서 누락된 컬럼: {missing_train_cols}")
        print(f"사용 가능한 컬럼: {train.columns.tolist()}")
        raise ValueError("필요한 컬럼이 누락되었습니다.")
    
    if missing_test_cols:
        print(f"❌ Test 데이터에서 누락된 컬럼: {missing_test_cols}")
        print(f"사용 가능한 컬럼: {test.columns.tolist()}")
        raise ValueError("필요한 컬럼이 누락되었습니다.")
    
    # 데이터 분할
    X = train[['title', 'full_text']]
    y = train['generated']
    
    print(f"✅ 피처 데이터 shape: {X.shape}")
    print(f"✅ 타겟 데이터 shape: {y.shape}")
    
    # Train-Validation 분할
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42
    )
    
    print(f"✅ 훈련 세트: {X_train.shape}")
    print(f"✅ 검증 세트: {X_val.shape}")
    print(f"✅ 훈련 세트 타겟 비율: {y_train.mean():.3f}")
    print(f"✅ 검증 세트 타겟 비율: {y_val.mean():.3f}")
    
except Exception as e:
    print(f"❌ 데이터 전처리 오류: {e}")
    print("데이터 구조를 다시 확인해주세요.")

# 개선된 TF-IDF Vectorization

In [ ]:
# 🚀 메모리 최적화된 TF-IDF 벡터화 (메모리 에러 해결)
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
import time

print("⚡ 메모리 절약 벡터화 시작...")
start_time = time.time()

get_title = FunctionTransformer(lambda x: x['title'], validate=False)
get_text = FunctionTransformer(lambda x: x['full_text'], validate=False)

# 메모리 절약을 위한 대폭 축소된 벡터화
vectorizer = FeatureUnion([
    ('title', Pipeline([
        ('selector', get_title),
        ('tfidf', TfidfVectorizer(
            ngram_range=(1,2), 
            max_features=800,       # 1200→800 (33% 감소)
            min_df=8,               # 5→8 (더 엄격한 필터링)
            max_df=0.8,             # 0.85→0.8 (더 엄격한 필터링)
            dtype='float32',        # 메모리 절약
            lowercase=True,         # 속도 향상
            stop_words=None         # 속도 향상
        ))
    ])),
    ('full_text', Pipeline([
        ('selector', get_text),
        ('tfidf', TfidfVectorizer(
            ngram_range=(1,2), 
            max_features=3000,      # 5000→3000 (40% 감소)
            min_df=8,               # 5→8 (더 엄격한 필터링)
            max_df=0.8,             # 0.85→0.8 (더 엄격한 필터링)
            dtype='float32',        # 메모리 절약
            lowercase=True,         # 속도 향상
            stop_words=None         # 속도 향상
        ))
    ]))
], n_jobs=1)  # 메모리 절약을 위해 병렬처리 비활성화

# 메모리 사전 정리
gc.collect()
print("🧹 사전 메모리 정리 완료")

# 피처 변환 (메모리 절약 모드)
print("📊 Title & Text 벡터화 중... (메모리 절약 모드)")
try:
    X_train_vec = vectorizer.fit_transform(X_train)
    print(f"✅ 훈련 데이터 벡터화 완료: {X_train_vec.shape}")
    
    # 중간 메모리 정리
    gc.collect()
    
    X_val_vec = vectorizer.transform(X_val)
    print(f"✅ 검증 데이터 벡터화 완료: {X_val_vec.shape}")
    
    # 시간 측정
    elapsed_time = time.time() - start_time
    print(f"✅ 전체 벡터화 완료! 소요시간: {elapsed_time:.1f}초")
    print(f"✅ 총 피처 차원: {X_train_vec.shape[1]} (메모리 최적화)")
    
    # 메모리 사용량 추정
    memory_usage_mb = (X_train_vec.shape[0] * X_train_vec.shape[1] * 4) / (1024*1024)  # float32 = 4bytes
    print(f"✅ 예상 메모리 사용량: ~{memory_usage_mb:.0f}MB")
    
    # 최종 메모리 정리
    gc.collect()
    print("🧹 벡터화 후 메모리 정리 완료")
    
except MemoryError as e:
    print(f"❌ 메모리 부족: {e}")
    print("💡 해결책:")
    print("1. 런타임 재시작 후 다시 시도")
    print("2. 더 작은 샘플로 테스트")
    print("3. 로컬 환경에서 실행")
    raise
except Exception as e:
    print(f"❌ 벡터화 오류: {e}")
    raise

MemoryError: 

# 베이스라인 모델 (비교용)

In [ ]:
# 🚀 메모리 최적화된 XGBoost (베이스라인)
import gc

print("🔍 시스템 환경 확인...")

# Colab에서 GPU 사용 확인
try:
    import subprocess
    gpu_info = subprocess.check_output(["nvidia-smi"], universal_newlines=True)
    print("✅ GPU 사용 가능")
    use_gpu = True
except:
    print("⚠️ GPU 사용 불가 - CPU 모드로 전환")
    use_gpu = False

# 메모리 절약 최적화된 설정
if use_gpu:
    xgb_baseline = XGBClassifier(
        n_estimators=50,        # 80→50 (메모리 절약)
        max_depth=3,            # 4→3 (메모리 절약)
        learning_rate=0.15,     # 0.1→0.15 (적은 트리로 보상)
        subsample=0.8,          # 메모리 절약
        colsample_bytree=0.8,   # 메모리 절약
        random_state=42,
        tree_method='gpu_hist',
        device='cuda:0',        
        max_bin=64,             # 128→64 (더 많은 메모리 절약)
        n_jobs=1                # GPU 사용 시 1로 설정
    )
    print("🚀 GPU 가속 모드 (메모리 최적화)")
else:
    xgb_baseline = XGBClassifier(
        n_estimators=50,        # 메모리 절약
        max_depth=3,            # 메모리 절약
        learning_rate=0.15,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=2                # CPU에서는 적당한 병렬처리
    )
    print("🖥️ CPU 모드 (메모리 최적화)")

print("\n🚀 베이스라인 모델 학습 시작...")
print(f"훈련 데이터 크기: {X_train_vec.shape}")

# 메모리 사전 정리
gc.collect()

try:
    # 모델 학습
    xgb_baseline.fit(X_train_vec, y_train)
    print("✅ 모델 학습 완료!")
    
    # 예측 및 평가
    val_probs_baseline = xgb_baseline.predict_proba(X_val_vec)[:, 1]
    auc_baseline = roc_auc_score(y_val, val_probs_baseline)
    
    print(f"\n📊 베이스라인 성능:")
    print(f"Validation AUC: {auc_baseline:.4f}")
    
    # GPU 메모리 정보 (가능한 경우)
    if use_gpu:
        try:
            gpu_mem = subprocess.check_output(["nvidia-smi", "--query-gpu=memory.used,memory.total", "--format=csv,noheader,nounits"], universal_newlines=True)
            print(f"GPU 메모리 사용량: {gpu_mem.strip()}")
        except:
            pass
    
    print("✅ 베이스라인 모델 완료!")
    
except Exception as e:
    print(f"❌ 모델 학습 오류: {e}")
    print("💡 해결책:")
    print("1. 런타임 재시작")
    print("2. 더 작은 데이터셋으로 테스트")
    print("3. CPU 모드로 전환")
    raise
finally:
    # 메모리 정리
    gc.collect()
    print("🧹 메모리 정리 완료")

# 하이퍼파라미터 튜닝된 모델

In [ ]:
# 🚀 메모리 절약형 하이퍼파라미터 튜닝 모델
import gc

print("🔧 개선된 모델 설정 중...")

# 메모리 사전 정리
gc.collect()

# 메모리 절약 + 성능 최적화 설정
if use_gpu:
    xgb_improved = XGBClassifier(
        n_estimators=80,            # 150→80 (메모리 절약)
        max_depth=4,                # 5→4 (메모리 절약)
        learning_rate=0.12,         # 0.1→0.12 (적은 트리로 보상)
        subsample=0.85,             # 0.8→0.85 (약간 증가)
        colsample_bytree=0.85,      # 0.8→0.85 (약간 증가)
        reg_alpha=0.1,              
        reg_lambda=1.0,             
        random_state=42,
        tree_method='gpu_hist',     # GPU 가속
        device='cuda:0',            # 새로운 GPU 설정
        max_bin=64,                 # 메모리 절약
        n_jobs=1                    # GPU 사용 시 1로 설정
    )
    print("🚀 GPU 가속 모드 (성능 최적화)")
else:
    xgb_improved = XGBClassifier(
        n_estimators=80,
        max_depth=4,
        learning_rate=0.12,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=2                    # CPU에서는 적당한 병렬처리
    )
    print("🖥️ CPU 모드 (성능 최적화)")

try:
    print("\n🚀 개선된 모델 학습 시작...")
    xgb_improved.fit(X_train_vec, y_train)
    print("✅ 개선 모델 학습 완료!")
    
    # 예측 및 평가
    val_probs_improved = xgb_improved.predict_proba(X_val_vec)[:, 1]
    auc_improved = roc_auc_score(y_val, val_probs_improved)
    
    print(f"\n📊 개선된 모델 성능:")
    print(f"Validation AUC: {auc_improved:.4f}")
    print(f"베이스라인 대비 향상: {auc_improved - auc_baseline:.4f}")
    
    if auc_improved > auc_baseline:
        print("🎉 성능 향상 달성!")
    else:
        print("🤔 베이스라인과 유사한 성능")
    
    print("✅ 하이퍼파라미터 튜닝 완료!")
    
except Exception as e:
    print(f"❌ 개선 모델 학습 오류: {e}")
    print("베이스라인 모델을 사용합니다.")
    # 오류 시 베이스라인 모델 사용
    xgb_improved = xgb_baseline
    val_probs_improved = val_probs_baseline
    auc_improved = auc_baseline
    
finally:
    # 메모리 정리
    gc.collect()
    print("🧹 메모리 정리 완료")

# 교차 검증으로 안정성 확인

In [ ]:
# 🔄 교차 검증 (메모리 절약 모드)
print("🤔 교차 검증을 실행하시겠습니까?")
print("⚠️ 주의: 교차 검증은 메모리를 많이 사용하고 시간이 오래 걸립니다.")

# 간단한 교차 검증 (메모리 절약)
run_cv = True  # False로 설정하면 교차 검증 건너뛰기

if run_cv:
    print("\n🔄 3-fold 교차 검증 시작... (메모리 절약)")
    
    try:
        from sklearn.model_selection import StratifiedKFold
        
        # 메모리 절약을 위해 3-fold로 축소
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        
        # 메모리 정리
        gc.collect()
        
        # 교차 검증 실행 (단일 스레드로 메모리 절약)
        print("📊 교차 검증 진행 중...")
        cv_scores = cross_val_score(
            xgb_improved, X_train_vec, y_train,  # 전체가 아닌 훈련 데이터만 사용
            cv=cv, scoring='roc_auc', n_jobs=1   # 메모리 절약을 위해 n_jobs=1
        )
        
        print(f"\n📊 교차 검증 결과:")
        print(f"CV AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        print(f"개별 fold 점수: {cv_scores}")
        
        # 검증 안정성 평가
        if cv_scores.std() < 0.01:
            print("✅ 모델이 안정적입니다 (낮은 표준편차)")
        elif cv_scores.std() < 0.02:
            print("🤔 모델이 적당히 안정적입니다")
        else:
            print("⚠️ 모델 안정성이 낮습니다 (높은 표준편차)")
            
    except Exception as e:
        print(f"❌ 교차 검증 오류: {e}")
        print("교차 검증을 건너뛰고 계속 진행합니다.")
        cv_scores = [auc_improved]  # 단일 검증 점수 사용
    
    finally:
        gc.collect()
        print("🧹 교차 검증 후 메모리 정리 완료")
        
else:
    print("⏭️ 교차 검증 건너뜀 (메모리 절약)")
    cv_scores = [auc_improved]  # 단일 검증 점수 사용

print(f"\n📊 최종 모델 안정성: {np.mean(cv_scores):.4f}")
print("✅ 교차 검증 단계 완료!")

# 성능 비교 시각화

In [ ]:
# 모델 성능 비교
models = ['베이스라인', '개선된 모델']
scores = [auc_baseline, auc_improved]

plt.figure(figsize=(8, 6))
bars = plt.bar(models, scores, color=['lightblue', 'orange'], alpha=0.7)
plt.ylim(0.85, max(scores) + 0.01)
plt.ylabel('Validation AUC')
plt.title('모델 성능 비교')
plt.grid(True, alpha=0.3)

# 점수 표시
for bar, score in zip(bars, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{score:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Inference (최고 성능 모델 사용)

In [ ]:
# 최종 모델로 전체 데이터 재훈련
print("🏆 최종 모델 준비 중...")

# 메모리 절약을 위한 최종 모델 설정
if use_gpu:
    final_model = XGBClassifier(
        n_estimators=100,           # 200→100 (메모리 절약)
        max_depth=5,                # 6→5 (메모리 절약) 
        learning_rate=0.12,         # 0.1→0.12 (적은 트리로 보상)
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        tree_method='gpu_hist',     # GPU 가속
        device='cuda:0',
        max_bin=64,                 # 메모리 절약
        n_jobs=1
    )
    print("🚀 GPU 가속 최종 모델")
else:
    final_model = XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.12,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=2                    # CPU에서는 적당한 병렬처리
    )
    print("🖥️ CPU 최종 모델")

try:
    print("\n🚀 전체 훈련 데이터로 최종 모델 학습...")
    
    # 메모리 정리
    gc.collect()
    
    # 전체 훈련 데이터로 벡터화 (메모리 절약 모드)
    print("📊 전체 데이터 벡터화 중...")
    X_full_vec = vectorizer.fit_transform(X)
    print(f"✅ 전체 데이터 벡터화 완료: {X_full_vec.shape}")
    
    # 메모리 중간 정리
    gc.collect()
    
    # 최종 모델 학습
    final_model.fit(X_full_vec, y)
    print("✅ 최종 모델 학습 완료!")
    
    # 테스트 데이터 전처리
    print("\n🔮 테스트 데이터 예측 중...")
    
    # 컬럼명 통일 (안전한 방식)
    if 'paragraph_text' in test.columns:
        test = test.rename(columns={'paragraph_text': 'full_text'})
        print("✅ 테스트 데이터 컬럼명 통일 완료")
    
    # 테스트 데이터 준비
    X_test = test[['title', 'full_text']].copy()
    print(f"📊 테스트 데이터 준비 완료: {X_test.shape}")
    
    # 메모리 정리
    gc.collect()
    
    # 테스트 데이터 벡터화
    X_test_vec = vectorizer.transform(X_test)
    print(f"✅ 테스트 데이터 벡터화 완료: {X_test_vec.shape}")
    
    # 예측 실행
    probs = final_model.predict_proba(X_test_vec)[:, 1]
    
    print(f"\n🎯 예측 완료!")
    print(f"✅ 예측값 범위: [{probs.min():.3f}, {probs.max():.3f}]")
    print(f"✅ 예측값 평균: {probs.mean():.3f}")
    print(f"🧠 현재 메모리 사용량: {get_memory_usage():.1f}MB")
    
except Exception as e:
    print(f"❌ 최종 예측 오류: {e}")
    print("💡 해결책:")
    print("1. 런타임 재시작 후 다시 시도")
    print("2. 더 작은 모델 파라미터 사용")
    print("3. 배치 처리로 나누어 예측")
    raise
    
finally:
    # 메모리 정리
    gc.collect()
    print("🧹 예측 후 메모리 정리 완료")

# Submission

In [ ]:
# Submission 파일 생성
try:
    # sample_submission 파일 읽기 (안전하게)
    if 'sample_submission' not in locals():
        sample_submission = read_csv_robust('sample_submission.csv')
    
    print(f"Sample submission shape: {sample_submission.shape}")
    print(f"Sample submission 컬럼: {sample_submission.columns.tolist()}")
    
    # 예측값 할당
    sample_submission['generated'] = probs
    
    print(f"예측값 통계:")
    print(f"- 최솟값: {probs.min():.4f}")
    print(f"- 최댓값: {probs.max():.4f}")
    print(f"- 평균값: {probs.mean():.4f}")
    print(f"- 표준편차: {probs.std():.4f}")
    
    # 파일 저장
    output_filename = 'improved_submission.csv'
    sample_submission.to_csv(output_filename, index=False)
    print(f"✅ 제출 파일 저장 완료: {output_filename}")
    
    # 파일 다운로드 (코랩에서)
    try:
        from google.colab import files
        files.download(output_filename)
        print(f"✅ 파일 다운로드 시작: {output_filename}")
    except ImportError:
        print("로컬 환경에서 실행 중입니다. 파일이 현재 디렉터리에 저장되었습니다.")
    
    # 성능 개선 요약
    print(f"\n=== 🎯 성능 개선 요약 ===")
    if 'auc_baseline' in locals() and 'auc_improved' in locals():
        print(f"베이스라인 AUC: {auc_baseline:.4f}")
        print(f"개선된 AUC: {auc_improved:.4f}")
        print(f"개선폭: +{auc_improved - auc_baseline:.4f}")
    if 'cv_scores' in locals():
        print(f"교차검증 AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    print(f"\n🔥 최종 모델 예측 완료!")
    
except Exception as e:
    print(f"❌ 제출 파일 생성 오류: {e}")
    print("이전 단계들이 모두 성공적으로 실행되었는지 확인해주세요.")